In [1]:
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from collections import defaultdict
from tqdm import tqdm_notebook as tqdm

In [2]:
# constants
k_b = 1.38064852e-23
m_proton = 1.6726219e-27
m_particle = m_proton * 4 # Helium
LG_sigma = 2.640 * 1e-10
LG_epsilon =  10.9 * k_b

# LG_sigma = 3.405 * 1e-10
# LG_epsilon =  119.8 * k_b
# m_particle = 0.03994 / 6.02e23  # Argon


In [3]:
unit_length = LG_sigma
unit_time = LG_sigma * np.sqrt(m_particle / LG_epsilon)
unit_temperature = LG_epsilon / k_b
unit_pressure = LG_epsilon / LG_sigma**3
unit_volume = LG_sigma**3

print(
    "unit_length: %0.2e" % unit_length,
    "\nunit_time: %0.2e"% unit_time,
    "\nunit_temperature: %0.2e" % unit_temperature,
    "\nunit_pressure: %0.2e" % unit_pressure,
    "\nunit_volume: %0.2e" % unit_volume
)

unit_length: 2.64e-10 
unit_time: 1.76e-12 
unit_temperature: 1.09e+01 
unit_pressure: 8.18e+06 
unit_volume: 1.84e-29


In [4]:
def norm(r):
    return np.sqrt(np.sum(r**2, axis=-1))

def calc_diff(r, box, with_nan=False, closest=True):
    """
    r is a Nx3 ndarray
    returns matrix NxNx: d
    d[i,j] = r[i] - r[j]
    note that this is antisymmetric matrix
    """
    r = r.copy()
    if closest is True:
        for i, bound in enumerate(box):
            r[:,i] = r[:,i] - (r[:, i] // bound)*bound
    
    D = np.zeros((r.shape[0], r.shape[0], 3))
    D += r[:, None, :]
    D -= r[None, :, :]
    
    if closest is True:
        for i, bound in enumerate(box):
            D[..., i][D[...,i] > bound/2] -= bound
            D[..., i][D[...,i] < -bound/2] += bound
    
    if with_nan:
        np.fill_diagonal(D[..., 0], np.nan)
        np.fill_diagonal(D[..., 1], np.nan)
        np.fill_diagonal(D[..., 2], np.nan)

    return D

def calc_dist(r, box):
    """
    r is a Nx3 ndarray
    returns matrix NxNx: d
    d[i,j] = ||r[i] - r[j]||
    note that this is symmetric matrix
    returns matrix of distancec
    """
    diff = calc_diff(r, box)
    return norm(diff)
    

## Lennard-Jones potential
$$V _ { \mathrm { LJ } } = 4 \varepsilon \left[ \left( \frac { \sigma } { r } \right) ^ { 12 } - \left( \frac { \sigma } { r } \right) ^ { 6 } \right]$$

$$V _ { \mathrm { LJ } } ^ \star = 4 \left[ \left( \frac { 1 } { r^\star } \right) ^ { 12 } - \left( \frac { 1 } { r^\star } \right) ^ { 6 } \right]$$

$$F_x  = 4 \varepsilon \left[ 12\left( \frac { \sigma } { r } \right) ^ { 12 } - 6\left( \frac { \sigma } { r } \right) ^ { 6 } \right] \frac{x}{r^2}$$
$$F_y  = 4 \varepsilon \left[ 12\left( \frac { \sigma } { r } \right) ^ { 12 } - 6\left( \frac { \sigma } { r } \right) ^ { 6 } \right] \frac{y}{r^2}$$
$$F_z  = 4 \varepsilon \left[ 12\left( \frac { \sigma } { r } \right) ^ { 12 } - 6\left( \frac { \sigma } { r } \right) ^ { 6 } \right] \frac{z}{r^2}$$


In [5]:
# parameters for helium
# https://www.seas.upenn.edu/~amyers/CSATalu.pdf
LG_sigma = 2.640 * 1e-10
LG_epsilon =  10.9 * k_b

def LJ_potential(r, box, closest=True):
    diff = calc_diff(r, box, with_nan=True, closest=closest)
    dist = norm(diff)
    
    one_over_r_power6 = (dist) ** (-6)
    one_over_r_power12 = one_over_r_power6 ** 2
    
    potential = 4 * (one_over_r_power12 - one_over_r_power6)
    
    np.fill_diagonal(potential, 0)
    return potential

def LJ_potential_derivative(r, box, closest=True):
    diff = calc_diff(r, box, with_nan=True, closest=closest)
    dist = norm(diff)
    
    one_over_r_power6 = (dist) ** (-6)
    one_over_r_power12 = one_over_r_power6 ** 2
    
    derivative = -4 * (12 * one_over_r_power12 - 
                      6 * one_over_r_power6) / dist
    np.fill_diagonal(derivative, 0)

    return derivative

def LJ_force(r, box, closest=True):
    """
    r: NxNx3 ndarray
    r[i,j]: distance vector from j to i, 
            i.e. r_i - r_j
    return F
    F[i,j] = force on i by j
    F[i,i] = 0
    """
    diff = calc_diff(r, box, with_nan=True, closest=closest)
    dist = norm(diff)
    
    one_over_r_power6 = (dist) ** (-6)
    one_over_r_power12 = one_over_r_power6 ** 2
    
    comon_part = 4 * (12 * one_over_r_power12 - 
                                6 * one_over_r_power6) / dist**2
    F = comon_part[..., None] * diff
    np.fill_diagonal(F[..., 0], 0)
    np.fill_diagonal(F[..., 1], 0)
    np.fill_diagonal(F[..., 2], 0)
    return F

In [6]:
def calc_acceleration(F):
    return np.sum(F, axis=1)

def system_energy(r, v, box):
    KE = 0.5 * np.sum(v**2)
    PE = np.sum(LJ_potential(r, box, closest=False))/2
    return KE, PE

## Distribution of speed vector
$$f _ { \mathbf { v } } \left( v _ { x } , v _ { y } , v _ { z } \right) = \left( \frac { m } { 2 \pi k T } \right) ^ { 3 / 2 } \exp \left[ - \frac { m \left( v _ { x } ^ { 2 } + v _ { y } ^ { 2 } + v _ { z } ^ { 2 } \right) } { 2 k T } \right]$$

In [7]:
def init_positions_on_grid(N, box):
    
    box_a, box_b, box_c = box
    
    box_vol = box_a * box_b * box_c
    d = (box_vol / N)**(1/3)
    pos_a = d/2 + np.arange(0, box_a-d+0.001, d)
    pos_b = d/2 + np.arange(0, box_b-d+0.001, d)
    pos_c = d/2 + np.arange(0, box_c-d+0.001, d)

    r_init = np.stack(np.meshgrid(pos_a, pos_b, pos_c), axis=-1).reshape(-1, 3)
    r_init += np.random.uniform(-d*0.05, d*0.05, r_init.shape)
    
    return r_init


def init_positions_random(N, box):
    
    box_vol = np.product(box)
    
    d = min(((box_vol / N)**(1/3))*0.80, 1)
    
    r_init = [np.random.uniform([0,0,0], box)]
    r_init_array = np.asarray(r_init)
    N-=1
    
    while N > 0:
        new_particle = np.random.uniform(np.asarray([0,0,0]) + d/2, np.asarray(box) - d/2)
        if min(norm(r_init_array - new_particle)) >= d:
            r_init.append(new_particle)
            r_init_array = np.asarray(r_init)
            N-=1
    return r_init_array

def init_velocity_boltzmann(N, temperature):
    v_mean = 0 * np.ones(3)
    v_sigma = temperature * np.eye(3)

    v_init = np.random.multivariate_normal(v_mean, v_sigma, size=N)
    v_init -= v_init.mean(axis=0)    

    return v_init

In [8]:
def step_ideal(r, v, a, t,dt,box):
    r += v * dt
    return r, v, a, round(t+dt, 6)

In [9]:
def step_LJ(r, v, a, t, dt, box):
    ###########################
    ## Stormer-Verlet Algorithm
    if a is None:
        a = calc_acceleration(
                LJ_force(r, box))
    
    r += v * dt + 0.5 * a * dt**2
    v += 0.5 * a * dt
    a = calc_acceleration(
        LJ_force(r, box))
    v += 0.5 * a * dt
    ###########################
    
#     r, v = put_in_box(r, v, box)
    return r, v, a, round(t+dt, 6)

### Start simulation

In [10]:
def simulate(r,v,t, box,
    history=None,
    iteration_time=1, dt=0.0005, record_interval=0.01,
    edit_system_function=None, edit_interval=None, edit_params=list()):
    """
    r,v,a,t: initial parameters of the system
    box: size of the box
    history: previous recordings, give if you wwant to continue simulation
    iteration_time: time to simulate
    dt: time interval of the one step
    record_interval: interval of recording the state of the system 
    """    
    
    if history is None:
        history = defaultdict(list)
        history["KE"].append(system_energy(r,v, box)[0])
        history["PE"].append(system_energy(r,v, box)[1])
        history["time"].append(t)
    else:
        r = history["rs"][-1].copy()
        v = history["vs"][-1].copy()
        t = history["time"][-1]
    
    edit_time = t
    
    a = calc_acceleration(LJ_force(r, box))
    
    
    for it in tqdm(range(int(iteration_time/dt)), mininterval=1):
    #     r,v,a,t,dp = step_ideal(r, v,a, t)
        r,v,a,t = step_LJ(r, v, a, t, dt, box)

        if t - history["time"][-1] >= record_interval - dt/4:
            KE,PE = system_energy(r, v, box)
            history["KE"].append(KE)
            history["PE"].append(PE)
            history["time"].append(round(history["time"][-1]+record_interval, 6))
            history["vs"].append(v.copy())
            history["rs"].append(r.copy())
        if edit_system_function is not None and t - edit_time >= edit_interval - dt/4:
            edit_time = round(edit_time+edit_interval, 6)
            (r,v,t) = edit_system_function((r,v,t), history, *edit_params)
        
    history["total"] = list(np.asarray(history["PE"]) + np.asarray(history["KE"]))
    return history

In [11]:
def estimmate_temperature(vs, box):
    vs = np.asarray(vs)
    KE_mean = np.sum(vs**2)/2/vs.shape[0]
    return KE_mean*2/(3 * vs.shape[1])

In [12]:
def pressure_viral(rs, vs, box):
    vs = np.asarray(vs)
    rs = np.asarray(rs)
    KE_mean = np.sum(vs**2)/2/vs.shape[0]
    W_int = np.mean([np.sum(LJ_potential_derivative(r, box) * calc_dist(r, box)) 
                     for r in rs])
    pressure = 1/np.product(box) * (KE_mean * (2/3) - 1/6 * W_int)
    return pressure